[Reference](https://tensorflow.google.cn/tutorials/wide)

These tutorial introdue how to implment feature column out of pandas dataframe

In [1]:
import tensorflow as tf
import pandas as pd
from pandas import DataFrame
import tempfile

## Explore Data

In [2]:
# import tempfile
# import urllib
# train_file = tempfile.NamedTemporaryFile()
# test_file = tempfile.NamedTemporaryFile()
# urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", train_file.name)
# urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", test_file.name)

In [3]:
import pandas as pd
train_file_name = "adult.data.txt"
test_file_name = "adult.test.txt"
CSV_COLUMNS = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "gender",
    "capital_gain", "capital_loss", "hours_per_week", "native_country",
    "income_bracket"]
df_train = pd.read_csv("adult.data.txt", names=CSV_COLUMNS, skipinitialspace=True)
df_test = pd.read_csv("adult.test.txt", names=CSV_COLUMNS, skipinitialspace=True, skiprows=1)

In [4]:
df_test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [5]:
df_test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


Transform the label

In [6]:
train_label = df_train["income_bracket"].apply(lambda x: ">50K" in x).astype(int)
test_label = df_test["income_bracket"].apply(lambda x: ">50K" in x).astype(int)

In [7]:
def input_fn(data_file, num_epochs, shuffle):
  """Input builder function."""
  df_data = pd.read_csv(
      tf.gfile.Open(data_file),
      names=CSV_COLUMNS,
      skipinitialspace=True,
      engine="python",
      skiprows=1)
  # remove NaN elements
  df_data = df_data.dropna(how="any", axis=0)
  labels = df_data["income_bracket"].apply(lambda x: ">50K" in x).astype(int)
  return tf.estimator.inputs.pandas_input_fn(
      x=df_data,
      y=labels,
      batch_size=100,
      num_epochs=num_epochs,
      shuffle=shuffle,
      num_threads=5)

# Selecting and Engineering feature for the model

A feature coloum can be one of these

* base feature column: raw column in original dataframe
* derived feature column: new column created base on some transformation over one or many raw column in original dataframe.

#### Base categorical column

In [8]:
gender = tf.feature_column.categorical_column_with_vocabulary_list(
    "gender", ["Female", "Male"])

In [9]:
# if we don't know possible value in advance
occupation = tf.feature_column.categorical_column_with_hash_bucket(
    "occupation", hash_bucket_size=1000)

In [10]:
education = tf.feature_column.categorical_column_with_vocabulary_list(
    "education", [
        "Bachelors", "HS-grad", "11th", "Masters", "9th",
        "Some-college", "Assoc-acdm", "Assoc-voc", "7th-8th",
        "Doctorate", "Prof-school", "5th-6th", "10th", "1st-4th",
        "Preschool", "12th"
    ])
marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    "marital_status", [
        "Married-civ-spouse", "Divorced", "Married-spouse-absent",
        "Never-married", "Separated", "Married-AF-spouse", "Widowed"
    ])
relationship = tf.feature_column.categorical_column_with_vocabulary_list(
    "relationship", [
        "Husband", "Not-in-family", "Wife", "Own-child", "Unmarried",
        "Other-relative"
    ])
workclass = tf.feature_column.categorical_column_with_vocabulary_list(
    "workclass", [
        "Self-emp-not-inc", "Private", "State-gov", "Federal-gov",
        "Local-gov", "?", "Self-emp-inc", "Without-pay", "Never-worked"
    ])
native_country = tf.feature_column.categorical_column_with_hash_bucket(
    "native_country", hash_bucket_size=1000)

#### Base continuous column

In [11]:
education_num = tf.feature_column.numeric_column("education_num")
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

#### Making continuous feature categorical by bucketization

In [12]:
age = tf.feature_column.numeric_column("age")
age_buckets = tf.feature_column.bucketized_column(
    age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

### Intersecting multiple column with CrossedColumn

In [13]:
education_x_occupation = tf.feature_column.crossed_column(
    ["education", "occupation"], hash_bucket_size=1000)

# Define Model

In [14]:
# first list all the feature column we defined 
base_columns = [
    gender, native_country, education, occupation, workclass, relationship,
    age_buckets,
]
crossed_columns = [
    tf.feature_column.crossed_column(
        ["education", "occupation"], hash_bucket_size=1000),
    tf.feature_column.crossed_column(
        [age_buckets, "education", "occupation"], hash_bucket_size=1000),
    tf.feature_column.crossed_column(
        ["native_country", "occupation"], hash_bucket_size=1000)
]

model_dir = tempfile.mkdtemp()
m = tf.estimator.LinearClassifier(
    model_dir=model_dir, feature_columns=base_columns + crossed_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_summary_steps': 100, '_model_dir': '/tmp/tmpgt4139gd', '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5}


In [15]:
model_dir = tempfile.mkdtemp()
m = tf.estimator.LinearClassifier(
    model_dir=model_dir, feature_columns=base_columns + crossed_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_save_summary_steps': 100, '_model_dir': '/tmp/tmp5s201ntv', '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5}


# Train Model

In [17]:
# set num_epochs to None to get infinite stream of data.
train_steps = 100
m.train(
    input_fn=input_fn(train_file_name, num_epochs=None, shuffle=True),
    steps=train_steps)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp5s201ntv/model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmp5s201ntv/model.ckpt.
INFO:tensorflow:Loss for final step: 30.485.


In [23]:
results = m.evaluate(input_fn=input_fn(test_file_name,num_epochs=1, shuffle=False))

INFO:tensorflow:Starting evaluation at 2017-09-10-00:48:41
INFO:tensorflow:Restoring parameters from /tmp/tmp5s201ntv/model.ckpt-100
INFO:tensorflow:Finished evaluation at 2017-09-10-00:48:43
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.835145, accuracy_baseline = 0.763774, auc = 0.881325, auc_precision_recall = 0.689125, average_loss = 0.355492, global_step = 100, label/mean = 0.236226, loss = 35.5078, prediction/mean = 0.229875


In [24]:
print("model directory = %s" % model_dir)
for key in sorted(results):
  print("%s: %s" % (key, results[key]))

model directory = /tmp/tmp5s201ntv
accuracy: 0.835145
accuracy_baseline: 0.763774
auc: 0.881325
auc_precision_recall: 0.689125
average_loss: 0.355492
global_step: 100
label/mean: 0.236226
loss: 35.5078
prediction/mean: 0.229875
